In [1]:
from collections import OrderedDict
from math import log, sqrt

import numpy as np
import pandas as pd
from pandas import DataFrame
from six.moves import cStringIO as StringIO

from bokeh.plotting import figure, show, output_file
import cv2

'''
When all values are determined, replace placeholders(10) and change the accuracy if needed.
'''

classifications = """
Classifiers,              true_positives, false_positives, true_negatives, false_negatives, accuracy
Gaussian Naive Bayes,          0,               286,            5114,              0,          >90
Logistic Regression,           195,              35,             877,             93,          <90
Support Vector Machine,        10,              10,             10,             10,            <90
Random Forest,                 10,              10,             10,             10,            >90
Voting,                        10,              10,             10,             10,            >90
"""

conf_matrix = OrderedDict([
    ("True Positive",   "#0d3362"),
    ("False Positive", "#c64737"),
    ("True Negative",     "black"  ),
    ("False Negative", "#62e72d"),
])

accuracy_color = OrderedDict([
    (">90", "#e69584"),
    ("<90", "#aeaeb8"),
])

df = pd.read_csv(StringIO(classification),
                 skiprows=1,
                 skipinitialspace=True,
                 engine='python',           
                )

In [2]:
width = 800
height = 800
inner_radius = 90
outer_radius = 300 - 10

minr = sqrt(log(.001 * 1E4))
maxr = sqrt(log(1000 * 1E4))
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

def rad(mic):
    return a * np.sqrt(np.log(mic * 1E4)) + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 9

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=(-420, 420), y_range=(-420, 420),
    min_border=0, outline_line_color="black",
    background_fill_color="#f0e1d2")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

In [3]:
# annular wedges

angles = []
for i in range(len(df)):
    angles.append(np.pi/2 - big_angle/2 - i*big_angle)
    
angles = np.array(angles)

colors = [accuracy_color[accuracy] for accuracy in df['accuracy']]
p.annular_wedge(
    0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df['true_positives']),
                -big_angle + angles + 7 * small_angle, -big_angle + angles + 8*small_angle,
                color = conf_matrix['True Positive'])
p.annular_wedge(0, 0, inner_radius, rad(df['false_positives']),
                -big_angle + angles + 5*small_angle, -big_angle + angles + 6*small_angle,
                color = conf_matrix['False Positive'])
p.annular_wedge(0, 0, inner_radius, rad(df['true_negatives']),
                -big_angle + angles + 3*small_angle, -big_angle + angles + 4*small_angle,
                color=conf_matrix['True Negative'])
p.annular_wedge(0, 0, inner_radius, rad(df['false_negatives']),
                -big_angle + angles + small_angle, -big_angle + angles+ 2*small_angle,
                color = conf_matrix['False Negative'])

GlyphRenderer(id='1050', ...)

In [4]:
# circular axes and lables
labels = np.power(10.0, np.arange(-3, 4))
radii = a * np.sqrt(np.log(labels * 1E4)) + b
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii[:-1], [str(r) for r in labels[:-1]],
       text_font_size="8pt", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-10, outer_radius+10,
                -big_angle+angles, -big_angle+angles, color="black")

# metric labels
xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2 + angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df['Classifiers'], angle=label_angle,
       text_font_size="9pt", text_align="center", text_baseline="middle")

GlyphRenderer(id='1072', ...)

In [ ]:
# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-40, -40], [-370, -390], color=list(accuracy_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=[gr for gr in acccuracy_color.keys()]+ '% Accuracy',
       text_font_size="7pt", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40, -40], [18, 0, -18, -36], width=30, height=13,
       color=list(conf_matrix.values()))
p.text([-15, -15, -15, -15], [18, 0, -18, -36], text=list(conf_matrix),
       text_font_size="9pt", text_align="left", text_baseline="middle")

output_file("MLiB.html", title="MLiB.py")

show(p)